In [ ]:
import json
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [ ]:
scores={}
for i in glob('*/scores.json'):
    print(os.path.dirname(i))
    with open(i,'r') as f:
        scores[os.path.dirname(i)]=json.load(f)

In [ ]:
dfmelt=[]
for k,v in scores.items():
    df=pd.DataFrame.from_dict(v).loc[['R2_train','R2_test']].rename(index={'R2_train':'Train','R2_test':'Test'})
    df.columns=[df.columns,[k]*len(df.columns)]
    dfmelt.append(df.melt(value_vars=df.columns.tolist(),ignore_index=False))
dfmelt=pd.concat(dfmelt).reset_index().rename(columns={'index':'Data','variable_0':'Model','variable_1':'Feature Set','value':"R$^{2}$"})

In [ ]:
scores

In [ ]:
{k:np.average([v['R2_train'],v['R2_test']]) for k,v in scores['ALL'].items()}

In [ ]:
learningcurvedf=pd.concat([dfmelt[dfmelt['Feature Set']==j].replace(j,float(j.split('_')[1])) for j in set([i for i in dfmelt['Feature Set'] if '16' in i])])

In [ ]:


g=sns.relplot(learningcurvedf,
            kind='line',
            x='Feature Set',
            y='R$^{2}$',
            hue='Model',
            col='Data',
            height=4, aspect=1, col_wrap=2,palette=sns.color_palette("Paired",9)
            )
# axes = g.axes.flatten()
# axes[0].axhline(10, ls='--')
# axes[1].axhline(30, ls='--')
# g.set_axis_labels("Training Set Ratio", "R$^{2}$")
g.set_titles("{col_var}={col_name}")
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 1))
g.set(xlim=(0.1, 0.8))
g.fig.suptitle(f"BSE 16 Features")
# plt.title(f"{name}")
plt.tight_layout()
plt.savefig(f'BSE_16_learning_curve.png',dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
g=sns.catplot(dfmelt.reset_index().sort_values(by=['Feature Set','Model']),
             kind='bar',
             x='Model',
              y='R$^{2}$',
              hue='Data',
              col='Feature Set',
              height=4, aspect=1, col_wrap=5,palette=sns.color_palette("Paired",2)
             )

# g.set_axis_labels("Training Set Ratio", "R$^{2}$")
g.set_titles("{col_var}={col_name}")
sns.move_legend(g, "upper left", bbox_to_anchor=(1, 1))
# g.set(ylim=(0, 1))
g.fig.suptitle(f"BSE Classical Testing")
# plt.title(f"{name}")
plt.tight_layout()
plt.savefig(f'BSE.png',dpi=300,bbox_inches='tight')
plt.show()